# Facial Recognition with Emotion Recognition and Liveness Detection

### Import necessary libraries

In [1]:
import cv2
import numpy as np
import dlib
import face_recognition
import numpy as np
from fer import FER
import smtplib
import datetime
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import pickle
import cv2

### Preprocessing

In [2]:
protoPath = r'face_detector\deploy.prototxt'
modelPath = r'face_detector\res10_300x300_ssd_iter_140000.caffemodel'
model = load_model('liveness.model')
le = pickle.loads(open('le.pickle', "rb").read())
net = cv2.dnn.readNetFromCaffe(protoPath, modelPath)
first_run = True
initial_time = datetime.datetime.utcnow()
s = smtplib.SMTP('smtp.gmail.com', 587)
s.starttls()
s.login("hetmehta61@gmail.com", "cufylheuvyfgmyru")

C:\Users\Het\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.preprocessing.label module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\Het\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.21.2 when using version 0.23.0. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


(235, b'2.7.0 Accepted')

In [3]:
# Connects to your computer's default camera
# 'http://192.168.0.100:8080/video' for remote else 0
string = 0
cap = cv2.VideoCapture(string)
het_image = face_recognition.load_image_file("Het_pic.jpg")
het_face_encoding = face_recognition.face_encodings(het_image)[0]
detector = FER()

known_face_encodings = [
    het_face_encoding
]

known_face_names = [
    "Het Mehta"
]

face_locations = []
face_encodings = []
face_names = []
process_this_frame = True
# Detect the coordinates
face_detector = dlib.get_frontal_face_detector()

### Capture video from source and perform the facial recognition along with emotion recognition and liveness detection

In [5]:
while True:
    ret, frame = cap.read()
    
    if string != 0:
        frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
        small_frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
    else:        
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector(gray)

    if len(faces) > 0: #Check if there are any faces in the frame        
        
        rgb_small_frame = small_frame[:, :, ::-1]

        if process_this_frame: # Process alternative frames to decrease the load on computer
            
            #Liveness detection
            (h, w) = frame.shape[:2]
            blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,(300, 300), (104.0, 177.0, 123.0))
            net.setInput(blob)
            detections = net.forward()

            for i in range(0, detections.shape[2]):
                # extract the confidence (i.e., probability) associated with the
                # prediction
                confidence = detections[0, 0, i, 2]

                # filter out weak detections
                if confidence > 0.5:
                    # compute the (x, y)-coordinates of the bounding box for
                    # the face and extract the face ROI
                    box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                    (startX, startY, endX, endY) = box.astype("int")

                    # ensure the detected bounding box does fall outside the
                    # dimensions of the frame
                    startX = max(0, startX)
                    startY = max(0, startY)
                    endX = min(w, endX)
                    endY = min(h, endY)

                    # extract the face ROI and then preproces it in the exact
                    # same manner as our training data
                    face = frame[startY:endY, startX:endX]
                    face = cv2.resize(face, (32, 32))
                    face = face.astype("float") / 255.0
                    face = img_to_array(face)
                    face = np.expand_dims(face, axis=0)

                    # pass the face ROI through the trained liveness detector
                    # model to determine if the face is "real" or "fake"
                    preds = model.predict(face)[0]
                    j = np.argmax(preds)
                    label = le.classes_[j]
                    print(label)
                if label == 'fake':
                    message = 'Subject: {}\n\n{}'.format('SECURITY ALERT FROM HOUSE', "Someone unauthorized is trying to enter the premises")
                    time_now = datetime.datetime.utcnow()
                    if(((time_now - initial_time).total_seconds() > 300) or first_run):
                        s.sendmail("hetmehta61@gmail.com", "hetmehta61@gmail.com", message)
                        print('Email Sent')
                        initial_time = time_now
                        first_run = False

            emotions = detector.detect_emotions(frame) # Detect emotions
            for i in emotions:
                emotion = i['emotions']
                dom_emotion = max(zip(emotion.values(), emotion.keys()))[1]
                print(dom_emotion)
                if(dom_emotion == 'angry' or dom_emotion == 'fear'): # If extreme emotions alert user
                    time_now = datetime.datetime.utcnow()
                    if(((time_now - initial_time).total_seconds() > 300) or first_run):
                        message = 'Subject: {}\n\n{}'.format('SECURITY ALERT FROM HOUSE', "Someone with extreme emotions detected")    
                        s.sendmail("hetmehta61@gmail.com", "hetmehta61@gmail.com", message)
                        print('Email Sent')
                        initial_time = time_now
                        first_run = False

            face_locations = face_recognition.face_locations(rgb_small_frame)
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

            face_names = []
            for face_encoding in face_encodings:
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                name = "Unknown"

                face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name = known_face_names[best_match_index]

                face_names.append(name)
    
        if 'Unknown' in face_names: # If an unknown person comes, it will alert the user
            if(((time_now - initial_time).total_seconds() > 300) or first_run):
                message = 'Subject: {}\n\n{}'.format('SECURITY ALERT FROM HOUSE', "Unknown individual at your premises")
                s.sendmail("hetmehta61@gmail.com", "hetmehta61@gmail.com", message)
                print('Email Sent')
                initial_time = time_now
                first_run = False

        process_this_frame = not process_this_frame


        for (top, right, bottom, left), name in zip(face_locations, face_names):
            if string != 0:
                top *= 2
                right *= 2
                bottom *= 2
                left *= 2
            else:
                top *= 4
                right *= 4
                bottom *= 4
                left *= 4

            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        cv2.imshow('Video', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        print('No faces found')
        cv2.imshow('Video', frame) 
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
s.quit()

real
neutral
real
neutral
real
neutral
real
neutral
real
neutral
real
neutral
real
neutral
real
neutral
real
neutral
real
neutral
fake
Email Sent
neutral
real
neutral
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
No faces found
fake
neutral
real
neutral
real
neutral
real
neutral
real
neutral
real
neutral
real
neutral
real
neutral
fake
neutral
real
neutral
real
neutral
real
neutral
No faces found
No faces found
fake
happy
fake
neutral
fake
neutral
fake
neutral
fake
neutral
fake
neutral
fake
neutral
fake
neutral
fake
neutral
No faces found
fake
neutral
real
neutral
real
neutral
real
neutral


(221,
 b'2.0.0 closing connection s16-20020a63ff50000000b003650ee901e1sm4802055pgk.68 - gsmtp')